In [1]:
import sys
sys.path.append("../../../h2o-py/build/main")

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
import h2o
from h2o.estimators import H2ODecisionTreeEstimator
h2o.init()

versionFromGradle='3.46.0',projectVersion='3.46.0.99999',branch='valenad-test-test',lastCommitHash='d7daf66d53941dbd75639a0ec6ffe1b2910de62a',gitDescribe='jenkins-3.46.0.2-1-gd7daf66d53',compiledOn='2024-05-24 14:11:07',compiledBy='potte'
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,11 mins 37 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.99999
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_potte_ej4obr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,23.91 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [4]:
survival_data = pd.read_csv('train-prep.csv')
survival_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
707,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,Q
708,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
709,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
710,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


# Applying split dataset to H2O

In [5]:
titanic_train = h2o.import_file('train-split.csv')
titanic_valid = h2o.import_file('valid-split.csv')
target_variable = 'Survived'
train_variables =['Age','Sex','Pclass']

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
valid = titanic_valid
train = titanic_train

In [7]:
def make_categorical(dataset):
    dataset[target_variable] = dataset[target_variable].asfactor()
    dataset["Sex"] = dataset["Sex"].asfactor()

In [8]:
make_categorical(train)
make_categorical(valid)

In [9]:
sdt_h2o = H2ODecisionTreeEstimator(model_id="decision_tree.hex",min_rows=1, max_depth=5,seed =168)
sdt_h2o.train(y=target_variable,x = train_variables,training_frame=train)

dt Model Build progress: |███████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODecisionTreeEstimator : Decision Tree
Model Key: decision_tree.hex

No summary for this model

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [10]:
predictions = sdt_h2o.predict(valid)
predictions

dt prediction progress: |████████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.869281,0.130719
0,0.537313,0.462687
0,0.647059,0.352941
1,0,1
0,0.869281,0.130719
1,0,1
0,0.943396,0.0566038
0,0.537313,0.462687
0,0.537313,0.462687
0,0.537313,0.462687


In [11]:
valid = valid.as_data_frame()

D:\skola\dip\h2o-3\_valenad-notebooks\machine-learning-experiments\Titanic_H2O-SkLearn\../../../h2o-py/build/main\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [12]:
train = train.as_data_frame()

In [13]:
predictions = predictions['predict'].as_data_frame()

In [14]:
valid = valid['Survived'] 

In [15]:
H2O_score=accuracy_score(valid, predictions)

# Applying split dataset to SkLearn

In [16]:
def Sex_to_binary(data):
    data['Sex'].replace('female', 0,inplace=True)
    data['Sex'].replace('male', 1,inplace=True)

In [17]:
train_data = pd.read_csv('train-split.csv')
train_data = train_data[['Sex', 'Age','Pclass','Survived']]
X_train = train_data.drop(columns =['Survived']) 
y_train = train_data['Survived']

valid_data = pd.read_csv('valid-split.csv')
valid_data = valid_data[['Sex', 'Age','Pclass','Survived']]
X_valid = valid_data.drop(columns =['Survived']) 
y_valid = valid_data['Survived']

In [18]:
Sex_to_binary(X_valid)
Sex_to_binary(X_train)

In [19]:
model = DecisionTreeClassifier(max_depth = 5)
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
Sklearn_score=accuracy_score(y_valid, predictions)

In [20]:
model

DecisionTreeClassifier(max_depth=5)

In [21]:
Sklearn_score

0.7741935483870968

In [22]:
H2O_score

0.7649769585253456